### Quickstart

* [hopsworks-tutorial quickstart](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/quickstart.ipynb)

In [6]:
!pip3 install -U hopsworks --quiet

In [7]:
!pip3 install joblib 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 7.8 MB/s eta 0:00:00


In [9]:
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 6.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.1 MB/s eta 0:00:0000:0100:01m


In [12]:
!pip3 install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.2 MB/s eta 0:00:00


In [14]:
!pip3 install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.1 MB/s eta 0:00:0000:0100:01


In [18]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 7.2 MB/s eta 0:00:0000:010:01


In [26]:
import joblib
import os
import time

import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
from math import radians

import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score 
#from scikit-learn import confusion_matrix
#from scikit-learn import f1_score

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

In [32]:
# Specify the window length as "4h"
window_len = "4h"

# Specify the URL for the data
url = "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_data/"

In [56]:
import requests
import io,urllib3
http = urllib3.PoolManager()
# Read the 'credit_cards.csv' file
s = requests.get(url + "credit_cards.csv").content
credit_cards_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
#credit_cards_df = pd.read_csv(url + "credit_cards.csv")

# Read the 'profiles.csv' file
# Parse the 'birthdate' column as dates
s2 = requests.get(url + "profiles.csv").content

profiles_df = pd.read_csv(io.StringIO(s2.decode('utf-8')), parse_dates=["birthdate"])
#profiles_df = pd.read_csv(url + "profiles.csv", parse_dates=["birthdate"])

# Read the 'transactions.csv' file
# Parse the 'datetime' column as dates
#s3 = requests.get(url +  "transactions.csv") 

#print(s3.decode('utf-8'))
#resp_text = urllib3.request.urlopen(url + "transactions.csv").read().decode('UTF-8')
#resp_text = http.request("GET",url + "transactions.csv")
#print(resp_text )


#trans_df = pd.read_csv(io.StringIO(s3.decode('utf-8')), parse_dates=["datetime"])
#trans_df = pd.read_csv(url + "transactions.csv", parse_dates=["datetime"])

# Display the first 3 rows of the 'transactions.csv' DataFrame
#trans_df.head(3)

###  Feature Engineering

In [57]:
# Merge the 'trans_df' DataFrame with the 'profiles_df' DataFrame based on the 'cc_num' column
age_df = trans_df.merge(profiles_df, on="cc_num", how="left")

# Compute the age at the time of each transaction and store it in the 'age_at_transaction' column
trans_df["age_at_transaction"] = (age_df["datetime"] - age_df["birthdate"]) / np.timedelta64(365, "D")

# Merge the 'trans_df' DataFrame with the 'credit_cards_df' DataFrame based on the 'cc_num' column
card_expiry_df = trans_df.merge(credit_cards_df, on="cc_num", how="left")

# Convert the 'expires' column to datetime format
card_expiry_df["expires"] = pd.to_datetime(card_expiry_df["expires"], format="%m/%y")

# Compute the days until the card expires and store it in the 'days_until_card_expires' column
trans_df["days_until_card_expires"] = (card_expiry_df["expires"] - card_expiry_df["datetime"]) / np.timedelta64(1, "D")

# Display the 'age_at_transaction' and 'days_until_card_expires' columns for the first few rows
trans_df[["age_at_transaction", "days_until_card_expires"]].head()

NameError: name 'trans_df' is not defined